## <u style="color: blue">7. Gridsearch</u> 

Comme précisé dans le notebook principal, vous trouverez ici les calculs de Gridsearch pour définir le modèle le plus performant pour la prédiction de tweets concernant des catastrophes:
- Decision Tree
- Random Forest
- Linear Regression
- XGBoost
- SVM

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from Decision_Tree import DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb

In [2]:
df_train = pd.read_csv('./CSV/train_tweets_cleaned.csv')

In [3]:
df_train.head()

,id,text,target,transformed_text,stemmed_text
0,1,Deeds Reason ALLAH Forgive,1,"['Deeds', 'Reason', 'ALLAH', 'Forgive']",deed reason allah forgiv
1,4,Forest fire near La Ronge Sask Canada,1,"['Forest', 'fire', 'near', 'La', 'Ronge', 'Sas...",forest fire near la rong sask canada
2,5,residents asked shelter place notified officer...,1,"['residents', 'asked', 'shelter', 'place', 'no...",resid ask shelter place notifi offic evacu she...
3,6,receive evacuation orders California,1,"['receive', 'evacuation', 'orders', 'California']",receiv evacu order california
4,7,sent photo Ruby smoke pours school,1,"['sent', 'photo', 'Ruby', 'smoke', 'pours', 's...",sent photo rubi smoke pour school


In [4]:
vectorizer = TfidfVectorizer(max_features=6678)
X = vectorizer.fit_transform(df_train['stemmed_text'].fillna('')).toarray()

y = df_train['target']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

Gridsearch pour le decision tree :

In [6]:
models_params = [
{
    'model': DecisionTreeClassifier(),
    'params': {
    'criterion': ['gini', 'entropy']
    },
},
{
    'model': RandomForestClassifier(),
    'params': {
    'criterion': ['gini', 'entropy']
    },
},
{
    'model': xgb.XGBClassifier(),
    'params': {
    },
},
{
    'model': svm.SVC(),
    'params': {
    'kernel': ['rbf', 'poly', 'sigmoid']
    },
},
{
    'model': LogisticRegression(),
    'params': {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    },
}]

In [7]:
scores = []

for model_param in models_params:
    clf = GridSearchCV(model_param['model'], model_param['params'], n_jobs=-1, verbose=2)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_param['model'],
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits


Matrice de confusion pour chaque modèle testé:

In [9]:
# confusion matrix for each models
for i in scores:
    model = i['model']
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('model: ', model)
    print('confusion matrix: ')
    print(confusion_matrix(y_test, y_pred))
    print('classification report: ')
    print(classification_report(y_test, y_pred))
    print('accuracy: ', accuracy_score(y_test, y_pred))
    print()

model:  DecisionTreeClassifier()
confusion matrix: 
[[572 220]
 [182 362]]
classification report: 
              precision    recall  f1-score   support

           0       0.76      0.72      0.74       792
           1       0.62      0.67      0.64       544

    accuracy                           0.70      1336
   macro avg       0.69      0.69      0.69      1336
weighted avg       0.70      0.70      0.70      1336

accuracy:  0.6991017964071856

model:  RandomForestClassifier()
confusion matrix: 
[[685 107]
 [210 334]]
classification report: 
              precision    recall  f1-score   support

           0       0.77      0.86      0.81       792
           1       0.76      0.61      0.68       544

    accuracy                           0.76      1336
   macro avg       0.76      0.74      0.75      1336
weighted avg       0.76      0.76      0.76      1336

accuracy:  0.7627245508982036

model:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              col

Le meilleur modèle est le SVM avec un score de 0.77 et les paramètres suivants : {'kernel': 'rbf'}

In [10]:
model = svm.SVC()
params = {
    'kernel': ['rbf', 'sigmoid'],
    'C': [0.1, 100],
    'gamma': [0.1, 0.01]
}

models_scores = []

clf = GridSearchCV(model, params, cv=2, n_jobs=-1, verbose=2)
clf.fit(X_train, y_train)
models_scores.append({
    'model': model,
    'best_score': clf.best_score_,
    'best_params': clf.best_params_
})

Fitting 2 folds for each of 8 candidates, totalling 16 fits


In [11]:
for i in models_scores:
    print('model: ', i['model'])
    print('score: ', i['best_score'])
    print('parameters: ', i['best_params'])
    print()

model:  SVC()
score:  0.7570198427555223
parameters:  {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid'}



In [8]:
# confusion matrix
cm = confusion_matrix(y_test, clf.predict(X_test))
print(cm)

NameError: name 'clf' is not defined

Maintenant que nous en avons terminé avec la partie du Gridsearch, nous pouvons retourner sur le premier notebook afin de mettre en application les résultats obtenus.

## <u style="color: blue">[8. Evaluation des modèles](./Eye_Of_Emergency.ipynb#8.-Evaluation-des-modèles)</u>